In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]='0,1,2,3'
import sys
import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('-mode', type=str, help='rgb or flow')
# parser.add_argument('-save_model', type=str)
# parser.add_argument('-root', type=str)

# args = parser.parse_args()


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms
import videotransforms


import numpy as np

from pytorch_i3d import InceptionI3d
from pytorch_i3d import FusionNet

# from charades_dataset import Charades as Dataset
from visual_tactile_dataset import VisualTactile as Dataset
import cv2
import matplotlib.pyplot as plt

In [ ]:
path = "/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/"
i3d = InceptionI3d(400, in_channels=2)
i3d.load_state_dict(torch.load('models/flow_imagenet.pt'))
i3d.train(True)

train_transforms = transforms.Compose([videotransforms.RandomCrop(224),
                                       videotransforms.RandomHorizontalFlip(),])
# train_transforms = transforms.Compose([videotransforms.CenterCrop(480),
#                                        videotransforms.RandomHorizontalFlip(),])
dataset = Dataset('train.txt',path, transform=train_transforms)
# dataset = Dataset(path)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0, pin_memory=False)
# for data in dataset:
c = 0
for data in dataloader:
    print("ok getting single data")
    f_vid, l_vid, tactile, pos, label = data
    c += 1
    if (c%2 == 0):
        break

# image,tactile,control_label,front_video,left_video,pos,label = dataset[0]

In [ ]:
print(f_vid.shape)
plt.imshow(f_vid[0][0][0])
# torch.from_numpy(left_video.transpose([3,0,1,2])).size()
# np.array(front_video[0]).shape

In [ ]:
path = "/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/Cheez/50_432/top/0/front_0.jpg"
image = cv2.imread(path)
# cv2.imshow('im', image)
# image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
# cv2.imshow('im',image)
# cv2.waitKey(1000)
np.asarray(image, dtype=np.float32)


In [ ]:
train_transforms = transforms.Compose([videotransforms.RandomCrop(224),
                                       videotransforms.RandomHorizontalFlip(),])

In [ ]:
def run(init_lr=0.1, max_steps=64e3, mode='rgb', root="/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/"
        , train_split=None, batch_size=1, save_model=''):
#     # setup dataset
#     train_transforms = transforms.Compose([videotransforms.RandomCrop(224),
#                                            videotransforms.RandomHorizontalFlip(),
#     ])
#     test_transforms = transforms.Compose([videotransforms.CenterCrop(224)])

    dataset = Dataset(root)
    dataset = Dataset(root,transform=videotransforms.CenterCrop(224))
#     dataset = Dataset(train_split, 'training', root, mode, train_transforms)
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=36, pin_memory=True)

#     val_dataset = Dataset(train_split, 'testing', root, mode, test_transforms)
#     val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=36, pin_memory=True)    

#     dataloaders = {'train': dataloader, 'val': val_dataloader}
#     datasets = {'train': dataset, 'val': val_dataset}

    
    # setup the model
    if mode == 'flow':
        i3d = InceptionI3d(400, in_channels=2)
        i3d.load_state_dict(torch.load('models/flow_imagenet.pt'))
    else:
        i3d = InceptionI3d(400, in_channels=3)
        i3d.load_state_dict(torch.load('models/rgb_imagenet.pt'))
    i3d.replace_logits(157)
#     #i3d.load_state_dict(torch.load('/ssd/models/000920.pt'))
#     i3d.cuda()
#     i3d = nn.DataParallel(i3d)

    lr = init_lr
    optimizer = optim.SGD(i3d.parameters(), lr=lr, momentum=0.9, weight_decay=0.0000001)
    lr_sched = optim.lr_scheduler.MultiStepLR(optimizer, [300, 1000])

    num_steps_per_update = 4 # accum gradient
    steps = 0
    # train it
    while steps < max_steps:#for epoch in range(num_epochs):
        print ('Step {}/{}'.format(steps, max_steps))
        print ('-' * 10)

#         # Each epoch has a training and validation phase
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 i3d.train(True)
#             else:
#                 i3d.train(False)  # Set model to evaluate mode
        i3d.train(True)
        tot_loss = 0.0
        tot_loc_loss = 0.0
        tot_cls_loss = 0.0
        num_iter = 0
        optimizer.zero_grad()

        # Iterate over data.
#         for data in dataloaders[phase]:
        for data in dataset:
            num_iter += 1
            # get the inputs
#             inputs, labels = data
            image,tactile,control_label,front_video,left_video,pos,label = data

#             # wrap them in Variable
#             inputs = Variable(inputs.cuda())
#             t = inputs.size(2)
#             labels = Variable(labels.cuda())
            front_video = front_video.unsqueeze(0)
            inputs = Variable(front_video)
#             inputs = torch.FloatTensor(front_video)
#             inputs = Variable(inputs)
            print(type(inputs))
            print(inputs.size())
#             inputs = front_video
            t = len(inputs)
            print(inputs.size())
            
            print("calculationg per_frame_logits")
            per_frame_logits = i3d(inputs)
            # upsample to input size
            print("upsampling per_frame_logits", per_frame_logits)
            per_frame_logits = F.upsample(per_frame_logits, t, mode='linear')

            print("computing localization loss")
            # compute localization loss
            loc_loss = F.binary_cross_entropy_with_logits(per_frame_logits, labels)
            tot_loc_loss += loc_loss.data[0]

            # compute classification loss (with max-pooling along time B x C x T)
            cls_loss = F.binary_cross_entropy_with_logits(torch.max(per_frame_logits, dim=2)[0], torch.max(labels, dim=2)[0])
            tot_cls_loss += cls_loss.data[0]

            loss = (0.5*loc_loss + 0.5*cls_loss)/num_steps_per_update
            tot_loss += loss.data[0]
            loss.backward()

            print ('{} Loc Loss: {:.4f} Cls Loss: {:.4f} Tot Loss: {:.4f}'.format(phase, tot_loc_loss/(10*num_steps_per_update), tot_cls_loss/(10*num_steps_per_update), tot_loss/10))
            if num_iter == num_steps_per_update and phase == 'train':
                steps += 1
                num_iter = 0
                optimizer.step()
                optimizer.zero_grad()
                lr_sched.step()
                if steps % 10 == 0:
                    print ('{} Loc Loss: {:.4f} Cls Loss: {:.4f} Tot Loss: {:.4f}'.format(phase, tot_loc_loss/(10*num_steps_per_update), tot_cls_loss/(10*num_steps_per_update), tot_loss/10))
                    # save model
                    torch.save(i3d.module.state_dict(), save_model+str(steps).zfill(6)+'.pt')
                    tot_loss = tot_loc_loss = tot_cls_loss = 0.
        if phase == 'val':
            print ('{} Loc Loss: {:.4f} Cls Loss: {:.4f} Tot Loss: {:.4f}'.format(phase, tot_loc_loss/num_iter, tot_cls_loss/num_iter, (tot_loss*num_steps_per_update)/num_iter) )
    


if __name__ == '__main__':
    # need to add argparse
#     run(mode=args.mode, root=args.root, save_model=args.save_model)
    run()


# Architecture after modified dataloader.

In [ ]:
# def run(init_lr=0.1, max_steps=64e3, mode='rgb', root='/ssd/Charades_v1_rgb', train_split='charades/charades.json', batch_size=8*5, save_model=''):
def run(init_lr=0.001, max_steps=10, mode='rgb', root='/media/pritesh/Entertainment/Visual-Tactile_Dataset/dataset/',\
        train_split='train.txt', test_split='test.txt', batch_size=1, save_model=''):
    # setup dataset
    train_transforms = transforms.Compose([videotransforms.RandomCrop(224),
                                           videotransforms.RandomHorizontalFlip(),])
    test_transforms = transforms.Compose([videotransforms.CenterCrop(224)])

    dataset = Dataset(train_split, root, mode, train_transforms)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

    val_dataset = Dataset(test_split, root, mode, test_transforms)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)    

    dataloaders = {'train': dataloader, 'val': val_dataloader}
    datasets = {'train': dataset, 'val': val_dataset}

    
    # setup the model
    if mode == 'flow':
        i3d = InceptionI3d(400, in_channels=2)
        i3d.load_state_dict(torch.load('models/flow_imagenet.pt'))
    else:
        i3d = InceptionI3d(400, in_channels=3)
        i3d.load_state_dict(torch.load('models/rgb_imagenet.pt'))
    i3d.replace_logits(157)
#     #i3d.load_state_dict(torch.load('/ssd/models/000920.pt'))
#     i3d.cuda()
#     i3d = nn.DataParallel(i3d)

    lr = init_lr
    optimizer = optim.SGD(i3d.parameters(), lr=lr, momentum=0.9, weight_decay=0.0000001)
#     lr_sched = optim.lr_scheduler.MultiStepLR(optimizer, [300, 1000])
    lr_sched = optim.lr_scheduler.MultiStepLR(optimizer, [4, 7])


    num_steps_per_update = 4 # accum gradient
    steps = 0
    # train it
    while steps < max_steps:#for epoch in range(num_epochs):
        print ('Step {}/{}'.format(steps, max_steps))
        print ('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                i3d.train(True)
            else:
                i3d.train(False)  # Set model to evaluate mode
                
            tot_loss = 0.0
            tot_loc_loss = 0.0
            tot_cls_loss = 0.0
            num_iter = 0
            optimizer.zero_grad()
            
            # Iterate over data.
            for data in dataloaders[phase]:
                num_iter += 1
                # get the inputs
                f_vid, l_vid, tactile, pos, labels = data

                # wrap them in Variable
                inputs = Variable(f_vid)
                t = inputs.size(2)
                labels = Variable(labels)
#                 inputs = Variable(inputs.cuda())
#                 t = inputs.size(2)
#                 labels = Variable(labels.cuda())
                print("go to i3d")
#                 per_frame_logits = i3d(inputs.float())
#                 # upsample to input size
#                 per_frame_logits = F.upsample(per_frame_logits, t, mode='linear')

#                 # compute localization loss
#                 loc_loss = F.binary_cross_entropy_with_logits(per_frame_logits, labels)
#                 tot_loc_loss += loc_loss.data[0]

#                 # compute classification loss (with max-pooling along time B x C x T)
#                 cls_loss = F.binary_cross_entropy_with_logits(torch.max(per_frame_logits, dim=2)[0], torch.max(labels, dim=2)[0])
#                 tot_cls_loss += cls_loss.data[0]

#                 loss = (0.5*loc_loss + 0.5*cls_loss)/num_steps_per_update
#                 tot_loss += loss.data[0]
#                 loss.backward()

#                 if num_iter == num_steps_per_update and phase == 'train':
#                     steps += 1
#                     num_iter = 0
#                     optimizer.step()
#                     optimizer.zero_grad()
#                     lr_sched.step()
#                     if steps % 10 == 0:
#                         print ('{} Loc Loss: {:.4f} Cls Loss: {:.4f} Tot Loss: {:.4f}'.format(phase, tot_loc_loss/(10*num_steps_per_update), tot_cls_loss/(10*num_steps_per_update), tot_loss/10))
#                         # save model
#                         torch.save(i3d.module.state_dict(), save_model+str(steps).zfill(6)+'.pt')
#                         tot_loss = tot_loc_loss = tot_cls_loss = 0.
#             if phase == 'val':
#                 print ('{} Loc Loss: {:.4f} Cls Loss: {:.4f} Tot Loss: {:.4f}'.format(phase, tot_loc_loss/num_iter, tot_cls_loss/num_iter, (tot_loss*num_steps_per_update)/num_iter) )
    


if __name__ == '__main__':
    # need to add argparse
#     run(mode=args.mode, root=args.root, save_model=args.save_model)
    run(save_model="test.pt")


In [32]:
i3d = InceptionI3d(num_classes=1)
i3d.load_state_dict(torch.load('video_i3dtrain000102.pt', map_location=lambda storage, loc: storage))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [78]:
i3d = InceptionI3d(num_classes = 1)
for para in i3d.parameters():
    para.requires_grad = False

In [79]:
from pytorch_i3d import FusionNet
fusedNet = FusionNet(i3d)

In [82]:
data =[]
for p in fusedNet.parameters():
#     print(p.requires_grad)
    data.append(p.requires_grad)
len(data)

177

In [65]:
def save_checkpoint(model, optimizer, scheduler, epoch):
    data = {'model_state' : model.state_dict(),
            'optimizer_state' : optimizer.state_dict(),
            'scheduler_state' : scheduler.state_dict(),
            'epoch' : epoch+1}
    torch.save(data, 'model_%d.tar' % (epoch + 1))
    
def load_model(input_channels, learning_rate, scheduler_list, checkpoint=None):
    i3d = InceptionI3d(1, in_channels=input_channels)
    fusedNet = FusionNet(i3d)
    #i3d = i3d.to(self.device)
    optimizer = optim.SGD(fusedNet.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0000001)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, scheduler_list)
    if (checkpoint is not None):
#         fusedNet.load_state_dict(torch.load(checkpoint, map_location=lambda storage, loc: storage))
        data = torch.load(checkpoint)
        fusedNet.load_state_dict(torch.load(data['model_state']))
        optimizer.load_state_dict(data['optimizer_state'])
        scheduler.load_state_dict(data['scheduler_state'])
    return fusedNet, optimizer, scheduler


# # save new model    
# optimizer = optim.SGD(i3d.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0000001)
# lr_sched = optim.lr_scheduler.MultiStepLR(optimizer, [50, 100, 150, 200])
# save_checkpoint(fusedNet, optimizer, lr_sched, 10)


In [72]:
a,b,c = load_model(3,0.1, [50, 100, 150, 200], 'model_11.tar')

AttributeError: 'collections.OrderedDict' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [22]:
import mmap
#load model
def load_big_file(f):
    with open(f, 'r+b') as f_in:
        #bf = io.BytesIO(f_in.read())
        # mmap seems to be much more memory efficient
        bf = mmap.mmap(f_in.fileno(), 0)
        f_in.close()
    return bf


# if torch.cuda.is_available():
#     f = cls.load_big_file(str(model_file))
#     state = torch.load(f)
# else:
#     f = cls.load_big_file(str(model_file))
#     state = torch.load(f, map_location={'cuda:0': 'cpu'})

def load_model(input_channels, learning_rate, scheduler_list, checkpoint=None):
    i3d = InceptionI3d(1, in_channels=input_channels)
    fusedNet = FusionNet(i3d)
    #i3d = i3d.to(self.device)
    optimizer = optim.SGD(fusedNet.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0000001)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, scheduler_list)
    if (checkpoint is not None):
#         fusedNet.load_state_dict(torch.load(checkpoint, map_location=lambda storage, loc: storage))
        f = load_big_file(checkpoint)
        data = torch.load(f, map_location={'cuda:0': 'cpu'})
        fusedNet.load_state_dict(torch.load(data['model_state']))
        optimizer.load_state_dict(data['optimizer_state'])
        scheduler.load_state_dict(data['scheduler_state'])
    return fusedNet, optimizer, scheduler

a,b,c = load_model(3, 0.1, [50, 100, 150, 200], 'model_1.tar')

RuntimeError: storage has wrong size: expected 0 got 1

# load model

In [117]:
sm = InceptionI3d(1, in_channels=3)
sm.replace_logits(1024)  #because by mistake i kept 1024 output in last logit
for para in sm.parameters():
    para.requires_grad = False
fusedNet = FusionNet(sm)
fusedNet = nn.DataParallel(fusedNet)
#i3d = i3d.to(self.device)
optimizer = optim.SGD(fusedNet.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0000001)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [50,100,150,200])

if torch.cuda.is_available():
    data = torch.load('model_51.tar')
else:
    data = torch.load('model_51.tar',map_location=lambda storage, loc: storage)
# data = torch.load('model_51.tar', map_location='cpu')
# sd = fusedNet.state_dict()
# ok = []
# for name, item in sd.items():
#     ok.append(name)
# len(ok)
fusedNet.load_state_dict(data['model_state'])
optimizer.load_state_dict(data['optimizer_state'])
scheduler.load_state_dict(data['scheduler_state'])
# for d in data:
#     print(d)


In [ ]:
fusedNet.train(True)

In [20]:
# Testing if lr_scheduler is working
i3d = InceptionI3d(1, in_channels=3)
fusedNet = FusionNet(i3d)
optimizer = optim.SGD(fusedNet.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0000001)
lr_sched = optim.lr_scheduler.MultiStepLR(optimizer, [50, 100, 150, 200])
for i in range(102):
    lr_sched.step()
lr_sched.get_lr()

[0.0001]

In [2]:
d = 40
save = "pritesj"
print("model_%d"  %d+ save + ".tar")

model_40pritesj.tar


In [30]:
1 == 1.0

True

In [29]:
print("-"*50, "\n")

-------------------------------------------------- 



# Read this
https://discuss.pytorch.org/t/runtimeerror-expected-4-dimensional-input-for-4-dimensional-weight-6-3-5-5-but-got-3-dimensional-input-of-size-3-256-256-instead/37189/4